In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
domain = pd.read_csv("./TripI/domain.csv")
domain_city_info = pd.read_csv("./TripI/domain_city_info.csv")
hotel_distance_to_location = pd.read_csv("./TripI/hotel_distance_to_location.csv")
hotel_info = pd.read_csv("./TripI/hotel_info.csv")
hotel_mapping = pd.read_csv("./TripI/hotel_mapping.csv")
hotel_v4 = pd.read_csv("./TripI/hotel_v4.csv")
hotel_v4_distinct = pd.read_csv("./TripI/hotel_v4_distinct.csv")
roothotel_info = pd.read_csv("./TripI/roothotel_info.csv")

C:\Users\ADMIN\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
hotel_image = pd.read_csv("./TripI/hotel_image.csv")
hotel_logging_detail = pd.read_csv("./TripI/Others/hotel_logging_detail.csv")
mobile_logging = pd.read_csv("./TripI/Others/mobile_logging.csv")
user_info = pd.read_csv("./TripI/Others/user_info.csv")

In [4]:
map = pd.read_csv('./TripI/out.csv')
map.head()

,domain_hotel_id,hotel_id,domain_id,similar_point
0,486861,9900,3,1.0
1,3356540,45306,3,1.0
2,6107639,35266,3,1.0
3,732184,23737,3,1.0
4,732142,4804,3,1.0


In [5]:
roothotel_info_sample = roothotel_info.drop(['logo', 'checkin_time','checkout_time', 'star_number','created_date_id','created_datetime'], axis =1)
roothotel_info_sample = roothotel_info_sample.rename(columns = {'id':'hotel_id'})
roothotel_info_sample.head(2)

,hotel_id,name,address,province_id,district_id,street_id,latitude,longitude,overall_score,description
0,1,OLIVE - Khách sạn Demo,"123231 Bắc Sơn, Quận Ba Đình, Hà Nội",11,15.0,28756.0,21.032986,105.839120,60.0,<p>Khách sạn Demo&nbsp;</p>\n
1,2,Khách sạn Nhật Linh,"194 Nguyễn Chí Thanh, Quận Hải Châu, Đà Nẵng",50,238.0,26881.0,16.064370,108.220542,84.0,<p style=text-align: justify;>Tọa lạc tại vị t...


In [6]:
hotel_mapping_sample = hotel_mapping.drop(['url', 'checkin_time','checkout_time','crawled_at', 'crawled_date_id','crawled_datetime'], axis =1)
hotel_mapping_sample = hotel_mapping_sample.rename(columns = {'domain_id':'domain_id1'})
hotel_OTA = hotel_mapping_sample.join(map.set_index('domain_hotel_id'), on ='domain_hotel_id')
hotel_OTA = hotel_OTA.rename(columns = {'name': 'name_OTA', 'address': 'address_OTA', 'star_number':'star_number_OTA','overall_score':'overall_score_OTA'})
hotel_OTA = hotel_OTA.drop(['domain_id1', 'longitude', 'latitude'], axis = 1)
hotel_OTA.head(2)

,id,domain_hotel_id,name_OTA,address_OTA,star_number_OTA,overall_score_OTA,hotel_id,domain_id,similar_point
0,00000c89-c5a3-4c0b-a02c-195d662a7cb7,1000000481682,Little Brick Saigon Hotel,"18 Bùi Thị Xuân, Phường Bến Thành, Phường Bến...",2,8.3,32377.0,2.0,0.908005
1,000405b3-6b57-4942-8838-89fb8fe4da17,9000000337368,La Hotel,"647 Tạ Quang Bửu, Phường 4, Quận 8, Phường 4, ...",2,-1.0,42652.0,2.0,0.747070


# 1. Hotel_map 

In [7]:
#hotel_map la bang chinh
hotel_map = hotel_OTA.join(roothotel_info_sample.set_index('hotel_id'), on='hotel_id')
hotel_map = hotel_map[hotel_map['similar_point']>0.95]
hotel_map = hotel_map.groupby(['domain_hotel_id']).agg(np.max)
hotel_map = hotel_map.reset_index()
hotel_map = hotel_map.rename(columns = {'id':'hotel_id_encoded'})
#hotel_map.T

In [8]:
#hotel_map.to_csv('./hotel_map.csv', index = False)

# 2. Map hotel_service with hotel_map

In [9]:
hotel_service= pd.read_csv("./TripI/hotel_service.csv")
hotel_service = hotel_service.rename(columns = {'hotel_id':'hotel_id_encoded'})
hotel_service = hotel_service.drop(['crawled_at', 'crawled_date_id', 'crawled_datetime'], axis = 1)
#idx = hotel_service.groupby(['id'])['crawled_datetime'].transform(max) == hotel_service['crawled_datetime']
#hotel_service = hotel_service[idx] 
#hotel_service['id'].value_counts()
#=> crawled_datetime nhu nhau het
hotel_service = hotel_service.groupby(['id']).agg(np.max)
temp1 = hotel_map[['hotel_id_encoded', 'hotel_id', 'domain_hotel_id']]
hotel_service = hotel_service.merge(temp1, on ='hotel_id_encoded')
hotel_service.T

,0,1,2,3,4,5,6,7,8,9,...,9441,9442,9443,9444,9445,9446,9447,9448,9449,9450
domain_id,2,2,3,2,5,5,2,3,3,5,...,4,2,5,3,2,3,3,3,4,3
hotel_id_encoded,f3a3af32-514c-426b-9837-c2eb0bba1227,86bb96d4-3927-40a3-ad66-cfaf7bec4d68,d77888ea-37b1-4f8e-86a8-f7b7f4b1996c,b1968212-4eb8-4c5a-a3e2-79a13b62bd5d,c115b28b-e1b1-49f3-af09-4256030210f0,f76ce3b0-59c1-4284-b61d-4af3f3c7dee6,c804857b-85ed-40da-be2c-f6fbe584acce,68e972e0-0afd-474e-9fc6-4e954d530356,aec87c81-acca-4c52-a6f0-b76a780ba3a9,8b7d1752-e709-4c96-9f1f-9471571cf2fb,...,5bf61522-767f-4e6b-83d3-e06b3136e88f,bdbed5da-164d-4113-89ac-ef697e08f7e1,a77ba786-4c2b-4204-a6b0-ad22bb33a8ae,3392459b-4e97-4d89-bb59-19813cb8bed9,91a9a840-242c-4c9b-86ea-48a1955305b8,a7bc7072-86be-4927-abbf-6e91774684fe,b52b0b8e-ea90-444b-b1e1-12f4f41e35e1,a4429302-b85f-4b79-a824-7254d5933ef0,aa7ef3f3-b705-4d40-8d91-484768c82ea2,6dc35543-8331-41ca-98a8-d21913731f5a
tours,0,1,1,0,1,0,0,1,1,0,...,-1,1,1,1,0,-1,1,-1,-1,-1
night_club,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
relax_spa,0,1,1,0,0,0,0,-1,-1,0,...,-1,0,0,-1,0,-1,-1,-1,-1,-1
relax_massage,-1,-1,-1,-1,1,0,-1,-1,-1,0,...,-1,-1,0,-1,-1,-1,-1,-1,-1,1
relax_steam_room,0,0,-1,0,-1,-1,0,-1,-1,-1,...,-1,0,-1,-1,0,-1,-1,-1,-1,-1
relax_outdoor_room,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
relax_outdoor_pool,0,0,-1,0,-1,-1,0,-1,-1,-1,...,-1,0,-1,-1,0,-1,-1,-1,-1,-1
relax_sauna,0,0,-1,0,-1,-1,0,-1,-1,-1,...,-1,0,-1,-1,0,-1,-1,-1,-1,-1


In [10]:
#hotel_service.to_csv('./hotel_service.csv', index = False)

In [11]:
#fig, axs = plt.subplots(nrows = 3, ncols= 3, figsize=(20, 10))
#sns.countplot(x = 'tours', data = hotel_service, ax=axs[0][0])
#sns.countplot(x = 'night_club', data = hotel_service, ax=axs[0][1])
#sns.countplot(x = 'relax_spa', data = hotel_service, ax=axs[0][2])
#sns.countplot(x = 'relax_massage', data = hotel_service, ax=axs[1][0])
#sns.countplot(x = 'relax_steam_room', data = hotel_service, ax=axs[1][1])
#sns.countplot(x = 'relax_outdoor_room', data = hotel_service, ax=axs[1][2])
#sns.countplot(x = 'relax_outdoor_pool', data = hotel_service, ax=axs[2][0])
#sns.countplot(x = 'relax_sauna', data = hotel_service, ax=axs[2][1])
#sns.countplot(x = 'relax_pool', data = hotel_service, ax=axs[2][2])

# 3. Map hotel_facility with hotel_map

In [12]:
hotel_quality = pd.read_csv("./TripI/hotel_quality.csv")
hotel_quality = hotel_quality.rename(columns = {'hotel_id':'hotel_id_encoded'})
hotel_quality = hotel_quality.drop(['crawled_at', 'crawled_date_id', 'crawled_datetime'], axis = 1)
#idx = hotel_quality.groupby(['id'])['crawled_datetime'].transform(max) == hotel_quality['crawled_datetime']
#hotel_quality = hotel_quality[idx] 
#hotel_quality['id'].value_counts()
#=> crawled_datetime nhu nhau het
hotel_quality = hotel_quality.groupby(['id']).agg(np.max)
temp1 = hotel_map[['hotel_id_encoded', 'hotel_id', 'domain_hotel_id']]
hotel_quality = hotel_quality.merge(temp1, on ='hotel_id_encoded')
hotel_quality.T

,0,1,2,3,4,5,6,7,8,9,...,9441,9442,9443,9444,9445,9446,9447,9448,9449,9450
domain_id,5,3,4,5,3,3,3,3,3,5,...,5,3,3,3,5,4,5,3,3,5
hotel_id_encoded,a5614d4c-7e1c-435a-ad06-273688783ab2,34f35f36-739e-4f92-a831-757d1b576a82,f0681a52-731a-4af8-801c-d3142dac9513,cb24f06a-27f2-4685-a10f-1f5bc100bb27,b8acdc40-5207-4205-ab05-50c0dc07c078,ea631b99-42e1-42ae-ace3-c1a118bd21ef,9312d54e-724c-4671-a3ee-653a878648db,217297e7-82ec-4802-98b4-f5c80a54be75,9f3d4d30-7036-440e-bd57-2b7aa3aaa2a7,d1fa7b66-0ff5-4979-b51a-26e88ab2d8db,...,2a8b4a39-8eb0-4724-b1b1-b08bbc9ae6bc,9dcd68de-c8c9-4e31-af98-03108d381367,72afa1d9-8e41-4f3b-914a-3cb2b6953bfc,8082cdf9-de28-4093-8b7b-7c1ad5265414,962f3ad5-b2d6-4a57-bfb6-612f064c6d71,5700a1b1-c3cf-4d0c-a449-670f648d5703,e0297159-d156-4f84-aaf2-1519d84958f8,e3f4d488-e5ef-41be-9485-c5e7eea5ccfc,c2ad6671-641f-4082-b14e-4a319ece5130,93d58700-4708-472a-a813-36a4c91833ae
star_number,-1,4,3,-1,0,3,0,3,1,2,...,-1,2,1,3,-1,3,-1,0,0,-1
cleanliness_score,8.5,8.6,-1,7.6,9.4,5.6,8.1,8,-1,-1,...,-1,9.2,6.5,-1,5.7,-1,9.3,-1,7.9,9.3
meal_score,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
average_web_response_time,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
min_web_response_time,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
max_web_response_time,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
num_reviews,19,19,-1,254,22,41,17,6,-1,-1,...,-1,34,24,-1,837,-1,34,-1,36,28
overall_score,8.8,8.7,10,7.7,9.4,6.1,7.8,7.5,-1,-1,...,-1,9,7.2,-1,6.5,7.2963,8.7,-1,7.9,9.2


In [13]:
#hotel_quality.to_csv('./hotel_quality.csv', index = False)

In [14]:
hotel_facility = pd.read_csv("./TripI/hotel_facility.csv")
hotel_facility = hotel_facility.rename(columns = {'hotel_id':'hotel_id_encoded'})
hotel_facility = hotel_facility.drop(['crawled_at', 'crawled_date_id', 'crawled_datetime'], axis = 1)
#idx = hotel_facility.groupby(['id'])['crawled_datetime'].transform(max) == hotel_facility['crawled_datetime']
#hotel_facility = hotel_facility[idx] 
#hotel_facility['id'].value_counts()
#=> crawled_datetime nhu nhau het
hotel_facility = hotel_facility.groupby(['id']).agg(np.max)
temp1 = hotel_map[['hotel_id_encoded', 'hotel_id', 'domain_hotel_id']]
hotel_facility = hotel_facility.merge(temp1, on ='hotel_id_encoded')
hotel_facility.T

,0,1,2,3,4,5,6,7,8,9,...,9442,9443,9444,9445,9446,9447,9448,9449,9450,9451
domain_id,3,5,2,3,4,4,3,3,4,4,...,5,5,3,5,4,3,4,3,5,4
hotel_id_encoded,cbdb0ad9-1f0f-4bfc-992a-200c244e5211,f87a4f85-0aa3-42af-8b3f-f412090e340b,b40a4aeb-63e7-4176-b938-99b9a0997756,3a475052-3d80-4512-8f42-17130dc56e3d,b6aadcde-874c-4644-bdb0-2b4a9eb17367,dafb40d7-31e8-478e-ae0d-d65af0343e87,55979850-45a0-459b-a645-03ee10a01b03,969130ee-9af0-4391-9af8-e492302831aa,95274676-299d-43ca-96ae-85b500b33937,b7a413c6-ffb6-4152-9adc-c239286364d7,...,c404126d-0146-4b3e-a96b-2b568020ded2,66118665-3258-45ed-b55c-7fd97b03d8d9,c41e03fb-a2de-4b46-b829-8e27432223de,89266e83-bdb6-41d5-8f82-c1187a1e012d,52bafd87-38a3-421d-8d6c-faeb63a3f104,54599471-d4e4-4850-9587-bc79392e283a,7dc4686f-c91d-4f56-998d-14681e3e4f95,b651e16b-6d55-48e0-91c9-79d28c056fba,ecd511ba-ccbb-4ce5-a676-5faf920b33b3,28c1c437-0383-47c9-b1f0-bf0174a9569a
currency_exchange,-1,1,1,-1,-1,-1,-1,-1,-1,-1,...,0,0,-1,1,-1,1,-1,1,1,-1
room_service_24_hour,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,...,0,0,-1,0,-1,1,-1,-1,1,-1
elevator,-1,1,1,-1,-1,-1,-1,-1,-1,-1,...,0,0,-1,0,-1,-1,-1,-1,0,-1
safely_deposit_boxed,-1,1,1,1,-1,-1,-1,-1,-1,-1,...,0,0,-1,0,-1,1,-1,-1,0,-1
luggage_storage,-1,1,1,1,-1,-1,-1,-1,-1,-1,...,0,0,-1,1,-1,1,-1,1,1,-1
poolside_bar,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
airport_transfer,-1,1,1,-1,-1,-1,-1,-1,-1,-1,...,0,0,-1,0,-1,-1,-1,-1,1,-1
restaurants,-1,1,1,-1,-1,-1,-1,-1,-1,-1,...,0,1,-1,0,-1,-1,-1,-1,0,-1


In [15]:
#hotel_facility.to_csv('./hotel_facility.csv', index = False)

In [16]:
#fig, axs = plt.subplots(nrows = 3, ncols=5, figsize=(20, 10))
#sns.countplot(x = 'currency_exchange', data = hotel_facility, ax=axs[0][0])
#sns.countplot(x = 'room_service_24_hour', data = hotel_facility, ax=axs[0][1])
#sns.countplot(x = 'elevator', data = hotel_facility, ax=axs[0][2])
#sns.countplot(x = 'safely_deposit_boxed', data = hotel_facility, ax=axs[0][3])
#sns.countplot(x = 'luggage_storage', data = hotel_facility, ax=axs[0][4])
#sns.countplot(x = 'poolside_bar', data = hotel_facility, ax=axs[1][0])
#sns.countplot(x = 'airport_transfer', data = hotel_facility, ax=axs[1][1])
#sns.countplot(x = 'baby_sitting', data = hotel_facility, ax=axs[1][2])
#sns.countplot(x = 'facilities_for_disabled_guests', data = hotel_facility, ax=axs[1][3])
#sns.countplot(x = 'private_beach', data = hotel_facility, ax=axs[1][4])
#sns.countplot(x = 'front_desk_24_hour', data = hotel_facility, ax=axs[2][0])
#sns.countplot(x = 'bar', data = hotel_facility, ax=axs[2][1])
#sns.countplot(x = 'laundry_service', data = hotel_facility, ax=axs[2][2])
#sns.countplot(x = 'shuttle_room', data = hotel_facility, ax=axs[2][3])

# 4. Map hotel_location with hotel_map

In [17]:
hotel_location = pd.read_csv("./TripI/hotel_location.csv")
hotel_location = hotel_location.rename(columns = {'hotel_id':'hotel_id_encoded'}) 
temp = hotel_map[['hotel_id_encoded', 'domain_id', 'domain_hotel_id']]
hotel_location = hotel_location.merge(temp, on = 'hotel_id_encoded')
hotel_location = hotel_location[hotel_location['domain_id_x'] == hotel_location['domain_id_y']]
hotel_location = hotel_location.rename(columns ={'domain_id_y': 'domain_id'})
hotel_location = hotel_location.drop(['domain_id_x', 'crawled_at','crawled_date_id','crawled_datetime'], axis = 1)
hotel_location = hotel_location.drop_duplicates()
hotel_location = hotel_location.rename(columns = {'id':'location_id'})
hotel_location.T

,0,1,2,3,4,5,6,7,8,9,...,9450,9451,9452,9453,9454,9455,9456,9457,9458,9459
location_id,019a1822-ba6a-45a4-a01b-8f347d3a26f2,01a8daa3-9921-49de-9a92-938e6e31f892,025fe16a-9a58-468a-a567-aa292ee30b0a,02e67fd5-147d-4fe7-b54d-77f6845f1c49,036cff00-064e-4ea7-bfce-2c412c32c77d,03b9af20-bb66-4741-a3a3-dc77c2920caf,03ded17d-36b8-4244-8fa6-da116520b0f5,049fb75e-5a6b-4dff-b177-3ad99d2e6ea3,04b15255-f495-4d20-951e-ecfbc2d841b9,0526ec1a-faf4-4a40-b076-00d2184f62e7,...,fd90e105-4b2f-4688-8389-ccdfd7013de5,fe0a7d49-ecca-4c2c-96fc-023c1ad2307b,fe1b38b6-7858-46a4-af95-073c3002be32,fe3f3a5f-881a-4270-b54e-9d1027dc2ab0,fe7c3b05-0615-4229-a23c-4f885b663901,fea93901-7341-4fed-ad62-8b319e6dc63a,feaf6297-a68a-4111-9b20-82d92b65c0e2,ff5d0529-c6ad-41f5-be73-6bfe0ad4bc9f,ff81d62b-a6f4-460b-84d9-43ef629d0022,ff90556c-ba8c-4d24-b869-a747f990c28e
hotel_id_encoded,7082df71-d2ef-44a8-945b-59dcae1bddd8,e9982bdf-b45b-4818-82ff-7721a7e72758,bc286a6e-44a5-4c1c-9ec2-3b8da3e662ae,e6a5043f-4360-4635-83ab-8ef1fde5edc5,d3d3695b-608e-4cd0-9a2e-cc582922b620,03486763-9e50-488a-af11-95c1ee556d94,74af5379-7ccc-4a0a-a73c-5fca48beaef6,359cc646-00b9-4c46-97b7-7e260372dd85,5bd12787-38db-4f14-9080-9546bb04f513,a78085f3-8bfa-494f-8589-f19507159ff7,...,46076cc7-e359-4107-aae4-f06ed64cba69,dd4d3214-2bd3-467b-998a-1277bfe0e9d4,faf63966-3b26-445e-a3da-66654af502fe,feb9ba72-516a-429e-ad74-17ffaf67c801,f4fa06a6-61a2-41c4-a37b-06d2efb30502,71cbea7d-81c0-48de-9c0f-4c66e66a5c27,e496a033-cd7f-446f-9ad4-abc8af332aaf,ebc74e4a-8a04-4798-bbc0-60beaa4cd547,bad39e0b-349a-407f-8706-1e54f027731f,ecf0135f-208b-4f25-a85a-3359e4df98ab
address,"97/10 Nguyễn Tri Phương, Cẩm Nam, Hội An, Tỉnh...","93/3 Ly Thuong Kiet, Son Phong, Phường Cẩm Châ...","35-37 Hàng Trống, Phường Hàng Trống, Quận Ho...","Tổ 1, Thôn Thanh Đông, Làng Cẩm Thanh, Cẩm Tha...","80 Lý Nam Đế, Phường 8, Đà Lạt, Tỉnh Lâm Đồng,...","Nga 3 Su Pan, Su Pan, Sử Pán, Sa Pa, Tỉnh Lào ...","Hoàng Quốc Việt, Hùng Thắng, phường Bãi Cháy, ...","71 Nam Kỳ Khởi Nghĩa, Phường 1, Phường 1, Đà L...","844 Nguyễn Trung Trực, Phường An Hòa, An Hòa, ...","145/15A Đỗ Xuân Hợp, Phước Long B, Quận 9, Thà...",...,"25 Nguyễn Phúc Tần, Minh An, Hội An, Việt Nam","46 Nguyen Van To, Hoan Kiem, Quận Hoàn Kiếm, H...","45 Phan Bội Châu ward 1, Đà Lạt, Việt Nam","169 Dang Tien Dong, Trung Liet, Dong Da, Quận ...","LÔ 2,KQH PHẠM NGŨ LÃO,P3,TP ĐÀ LẠT, Đà Lạt, Vi...","Lô A5 KQH Phan Đình Phùng - Hai Bà Trưng, phườ...","Nghe Hamlet, Van Hoa Commune, Ba Vi District, ...",SỐ 6 - HUỲNH THÚC KHÁNG - PHƯỜNG 4 - ĐÀ LẠT ĐỐ...,"An Bang beach, An Tan, Cam An, Hoi An, Cẩm An,...","Ap Ong Lang, Xa Cua Duong, Ong Lang, Phú Quốc,..."
latitude,15.8728,15.8816,21.0305,15.8806,11.9632,22.2928,20.9553,11.9407,9.98159,10.8237,...,15.8754,21.0316,11.9446,21.0137,11.9382,11.9493,21.0826,11.9353,15.9125,10.2622
longitude,108.335,108.338,105.85,108.353,108.44,103.922,107.021,108.434,105.104,106.771,...,108.327,105.846,108.437,105.822,108.436,108.434,105.392,108.43,108.338,103.942
domain_id,2,2,2,2,2,2,2,2,2,2,...,5,5,5,5,5,5,5,5,5,5
domain_hotel_id,3000020013165,1000000651303,1000000324621,3000010032373,5000002864574,9000000733386,1000001059986,3000010026819,1000000531368,3000020018704,...,5648566,1905309,3452942,1526078,2642390,5251194,1485545,2669232,2293529,1637877


In [18]:
#hotel_location.to_csv('./hotel_location.csv', index = False)

In [19]:
hotel_distance_to_location = pd.read_csv("./TripI/hotel_distance_to_location.csv")
hotel_distance_to_location = hotel_distance_to_location.drop(['crawled_at','crawled_date_id','crawled_datetime'],axis = 1)
temp = hotel_location[['location_id','hotel_id_encoded','domain_id','domain_hotel_id']]
hotel_distance_to_location = hotel_distance_to_location.merge(temp, on ='location_id')
hotel_distance_to_location = hotel_distance_to_location.rename(columns ={'domain_id_y': 'domain_id'})
hotel_distance_to_location = hotel_distance_to_location.drop(['domain_id_x'], axis = 1)
hotel_distance_to_location.T

,0,1,2,3,4,5,6,7,8,9,...,111330,111331,111332,111333,111334,111335,111336,111337,111338,111339
id,00030722-a322-4af1-b00c-28f1f95c96b1,02b69b3c-7e02-4426-af6f-b37043e37be8,061dcd1c-88a7-40a0-a1ba-f0d7454f42b6,0e6ecfe6-2b0a-4724-8d9e-c8c27f240dca,13dc6104-a2ce-47d6-8413-4f58c082b404,2d05f791-58a9-4881-b84b-8ef58dcadbed,31d838fe-8557-454f-9a85-5b1d7abdf77f,3b0fe8c9-c514-485f-94a1-6f6445a280f7,51dfb764-335f-4ac2-8d3a-acea3be4b564,62f16b11-9ecb-4dff-ae61-d0f51a46b722,...,6f05be10-03bb-4565-b79d-8de4e32d9492,840ad2ea-8ecb-4fc8-a4ae-fa333070d766,d176aa7a-797f-41d4-8987-27faa80d1b3f,e1d58cd5-022a-40fc-8371-7f7602cc0972,53c39590-0490-483d-9d4d-d9cd220ac65f,76c4691b-6b1a-46ae-8c16-1e6327f3fc6e,832141aa-71bb-4337-a46e-7920b2cbe0af,963431e4-e325-43b0-b1a4-d057bfbabb64,bb754d02-e718-4be2-a07c-f53f20612dce,bbb42f92-c680-401c-bc10-17564fea5ba3
location_id,a2c4160a-07a6-4aa3-8c50-b4446cf2b447,a2c4160a-07a6-4aa3-8c50-b4446cf2b447,a2c4160a-07a6-4aa3-8c50-b4446cf2b447,a2c4160a-07a6-4aa3-8c50-b4446cf2b447,a2c4160a-07a6-4aa3-8c50-b4446cf2b447,a2c4160a-07a6-4aa3-8c50-b4446cf2b447,a2c4160a-07a6-4aa3-8c50-b4446cf2b447,a2c4160a-07a6-4aa3-8c50-b4446cf2b447,a2c4160a-07a6-4aa3-8c50-b4446cf2b447,a2c4160a-07a6-4aa3-8c50-b4446cf2b447,...,ec8e3e4b-2b93-4757-b998-5da7b2b230ff,ec8e3e4b-2b93-4757-b998-5da7b2b230ff,ec8e3e4b-2b93-4757-b998-5da7b2b230ff,ec8e3e4b-2b93-4757-b998-5da7b2b230ff,a4b19006-d419-4b74-b2e8-108b70920362,a4b19006-d419-4b74-b2e8-108b70920362,a4b19006-d419-4b74-b2e8-108b70920362,a4b19006-d419-4b74-b2e8-108b70920362,a4b19006-d419-4b74-b2e8-108b70920362,a4b19006-d419-4b74-b2e8-108b70920362
name,1900 Le Theatre,Sân bay Quốc tế Cát Bi,Cung Thiếu nhi Hà Nội,Đền Quán Thánh,Tháp BIDV,Gourmet Restaurant,Vietcombank Tower,Sân bay Quốc tế Nội Bài,Đền Ngọc Sơn,Nhà hát múa rối Thăng Long,...,Phia Đén,Pia Oac,Coc Bo Cave,Hồ Thang Hen,Cánh đồng lúa thảo dược đa sắc màu,Pu Mat National Park,Thác Hồng Sơn,Thác mưa,Thác Vình,Thanh An Tea Hills
distance,0.5,93.1,0.4,2.3,0.3,0.1,0.5,21,0.3,0.2,...,40.3517,40.7317,41.4282,10.6991,16.475,28.1714,28.0155,24.2112,28.7825,30.0926
distance_unit,km,km,km,km,km,km,km,km,km,km,...,KILOMETER,KILOMETER,KILOMETER,KILOMETER,KILOMETER,KILOMETER,KILOMETER,KILOMETER,KILOMETER,KILOMETER
duration,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
category,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,Ngắm cảnh & Địa điểm nổi tiếng,Thiên nhiên,Thiên nhiên,Thiên nhiên,Thiên nhiên,Công viên & Sở thú,Thiên nhiên,Thiên nhiên,Thiên nhiên,Thiên nhiên
hotel_id_encoded,7a0184a2-080c-4dbe-a20a-434a7fc24617,7a0184a2-080c-4dbe-a20a-434a7fc24617,7a0184a2-080c-4dbe-a20a-434a7fc24617,7a0184a2-080c-4dbe-a20a-434a7fc24617,7a0184a2-080c-4dbe-a20a-434a7fc24617,7a0184a2-080c-4dbe-a20a-434a7fc24617,7a0184a2-080c-4dbe-a20a-434a7fc24617,7a0184a2-080c-4dbe-a20a-434a7fc24617,7a0184a2-080c-4dbe-a20a-434a7fc24617,7a0184a2-080c-4dbe-a20a-434a7fc24617,...,9a1a314c-0441-4662-a5c1-eb4f2a03f714,9a1a314c-0441-4662-a5c1-eb4f2a03f714,9a1a314c-0441-4662-a5c1-eb4f2a03f714,9a1a314c-0441-4662-a5c1-eb4f2a03f714,da9031a9-d7a5-4645-ab21-f049c8067c99,da9031a9-d7a5-4645-ab21-f049c8067c99,da9031a9-d7a5-4645-ab21-f049c8067c99,da9031a9-d7a5-4645-ab21-f049c8067c99,da9031a9-d7a5-4645-ab21-f049c8067c99,da9031a9-d7a5-4645-ab21-f049c8067c99
domain_id,5,5,5,5,5,5,5,5,5,5,...,2,2,2,2,2,2,2,2,2,2
domain_hotel_id,2508137,2508137,2508137,2508137,2508137,2508137,2508137,2508137,2508137,2508137,...,9000000337039,9000000337039,9000000337039,9000000337039,1000000978852,1000000978852,1000000978852,1000000978852,1000000978852,1000000978852


In [20]:
#hotel_distance_to_location.to_csv('./hotel_distance_to_location.csv', index = False)

# 5. Map hotel_price_daily with hotel_map

In [21]:
hotel_price_daily = pd.read_csv("./TripI/hotel_price_daily.csv")
temp = hotel_map[['domain_hotel_id', 'domain_id','hotel_id_encoded','hotel_id']]
hotel_price_daily = hotel_price_daily.merge(temp, on = 'domain_hotel_id')
hotel_price_daily = hotel_price_daily[hotel_price_daily['domain_id_x'] == hotel_price_daily['domain_id_y']]
hotel_price_daily.head(5)
hotel_price_daily = hotel_price_daily.rename(columns ={'domain_id_y': 'domain_id'})
hotel_price_daily = hotel_price_daily. drop(['domain_id_x'], axis = 1)
#Tao bang hotel_price chua 3 cot nay o dong code duoi
#hotel_price_daily['final_amount_min_MIN'] = hotel_price_daily.groupby(['domain_hotel_id'])['final_amount_min'].transform(min)
#hotel_price_daily['final_amount_min_MAX'] = hotel_price_daily.groupby(['domain_hotel_id'])['final_amount_min'].transform(max)
#hotel_price_daily['final_amount_min_MEAN'] = hotel_price_daily.groupby(['domain_hotel_id'])['final_amount_min'].transform(np.mean)
hotel_price_daily

,domain_hotel_id,checkin_date_id,final_amount_min,domain_id,hotel_id_encoded,hotel_id
0,1000000171688,20200816,2317282,2.0,8ac1134b-cae9-4a90-b939-374ef3a9d2d9,8272.0
1,1000000171688,20200813,2672072,2.0,8ac1134b-cae9-4a90-b939-374ef3a9d2d9,8272.0
2,1000000171688,20200814,2325434,2.0,8ac1134b-cae9-4a90-b939-374ef3a9d2d9,8272.0
3,1000000171688,20200817,2663542,2.0,8ac1134b-cae9-4a90-b939-374ef3a9d2d9,8272.0
4,1000000171688,20200815,2317282,2.0,8ac1134b-cae9-4a90-b939-374ef3a9d2d9,8272.0
...,...,...,...,...,...,...
47729,3684511,20200819,350000,5.0,765bd2f8-cabd-4a5a-889a-aa8bb35b3759,33482.0
47730,4859836,20200819,405000,5.0,af7b319b-da05-418a-89cd-e507f7a16bd9,33667.0
47731,4863858,20200819,300000,5.0,c1e954b5-5b9b-4b50-b3f7-4fdf5c9c2610,42893.0
47732,4910971,20200819,450000,5.0,932b8d7f-746f-4b92-bada-75740e238e6c,38644.0


In [22]:
#hotel_price_daily.to_csv('./hotel_price_daily.csv', index = False)

# 6. Make hotel_price from hotel_price_daily

In [23]:
hotel_price = hotel_price_daily.groupby(['domain_hotel_id']).agg({'final_amount_min': 'mean'})
hotel_price['mean'] = hotel_price_daily.groupby(['domain_hotel_id']).agg({'final_amount_min': 'mean'})
hotel_price['min'] = hotel_price_daily.groupby(['domain_hotel_id']).agg({'final_amount_min': 'min'})
hotel_price['max'] = hotel_price_daily.groupby(['domain_hotel_id']).agg({'final_amount_min': 'max'})
hotel_price['hotel_id'] = hotel_price_daily.groupby(['domain_hotel_id']).agg({'hotel_id': 'max'})
hotel_price = hotel_price.reset_index()
hotel_price = hotel_price.drop(['final_amount_min'],axis =1)
hotel_price

,domain_hotel_id,mean,min,max,hotel_id
0,10960,1.660457e+06,1322400,1740000,9674.0
1,10963,1.584386e+06,1363996,1847474,761.0
2,10964,3.575098e+06,3544128,3754345,1180.0
3,10966,1.821575e+06,1817655,1824868,46671.0
4,10967,1.073593e+06,1073593,1073593,67.0
...,...,...,...,...,...
7259,9000000861564,6.485984e+05,646049,652406,45904.0
7260,9000000862281,7.639367e+05,715625,798923,47357.0
7261,9000000864662,2.369014e+05,236004,238327,47043.0
7262,9000000873213,5.979888e+05,595360,601816,47462.0


In [24]:
#hotel_price.to_csv('./hotel_price.csv', index = False)

# 7. Map hotel_review with hotel_map

In [25]:
hotel_review = pd.read_csv("./TripI/hotel_review.csv")
temp = hotel_map[['domain_hotel_id', 'domain_id','hotel_id_encoded' ]]
hotel_review = hotel_review.drop(['crawled_at'], axis = 1)
hotel_review = hotel_review.merge(temp, on = 'domain_hotel_id')
hotel_review = hotel_review[hotel_review['domain_id_x'] == hotel_review['domain_id_y']]
hotel_review = hotel_review.rename(columns ={'domain_id_y': 'domain_id'})
hotel_review = hotel_review. drop(['domain_id_x'], axis = 1)
hotel_review.head(3)

C:\Users\ADMIN\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (4,5,10,14) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,id,raw_id,domain_hotel_id,username,review_id,lang_code,review_datetime,title,text,travel_purpose,crawled_date_id,crawled_datetime,emotion,score,domain_id,hotel_id_encoded
0,c10b1ccf-5f8d-4420-a3e2-a8dfeab4bb2a,187c51d4-1070-44e1-984c-7f0d1cade8a1,3886064,Nhung,4465881281,en-gb,2019-02-11 18:15:40,Very pleasant stay in Hanoi Old Quarter,The very best hostel that I have stayed in Han...,leisure,20200821.0,2020-08-21 09:21:41,NaN,4.000000,5.0,ae59c20e-cda7-4d72-b783-e289bfa62731
1,7ee80ae3-ad53-411e-b2d1-c0d21ea20ef1,b7dc4ced-1dce-42b5-8bff-8b7602753295,3886064,Raf,1342211902,nl,2019-01-18 04:36:59,Rất tốt,"Ligging, hygiëne",leisure,20200821.0,2020-08-21 09:21:41,NaN,3.333333,5.0,ae59c20e-cda7-4d72-b783-e289bfa62731
2,bbf5c8d8-1250-4932-a2a9-8435f3d4bb68,b7dc4ced-1dce-42b5-8bff-8b7602753295,3886064,Bich,1503021633,fr,2019-01-30 09:44:18,Xuất sắc,"Bon prix , propre, hyper centre!",leisure,20200821.0,2020-08-21 09:21:41,NaN,4.000000,5.0,ae59c20e-cda7-4d72-b783-e289bfa62731


In [26]:
#hotel_review.to_csv('./hotel_review.csv', index = False)

# 8. Clean province, district, street

In [27]:
province = pd.read_csv("./TripI/province.csv")
province = province.drop(['country_id','latitude','longitude','northeast_lat','northeast_lng','southwest_lat','southwest_lng','distance_from_northeast','distance_from_southwest'], axis = 1)
province.head(3)

,id,name,code,name_no_accent,shortname,priority
0,1,Thừa Thiên - Huế,VN-26,thua thien hue,Huế,1
1,2,Kiên Giang,VN-47,kien giang,K.Giang,1
2,3,Hải Phòng,VN-62,hai phong,H.Phòng,0


In [28]:
district = pd.read_csv("./TripI/district.csv")
district = district.drop(['code','northeast_lat','northeast_lng','southwest_lat','southwest_lng','distance_from_northeast','distance_from_southwest'], axis = 1)
district.head(3)

,id,province_id,name,latitude,longitude,name_no_accent
0,1,1,Huyện A Lưới,16.230374,107.337579,huyen a luoi
1,2,2,Huyện An Biên,9.806456,105.055742,huyen an bien
2,3,3,Huyện An Dương,20.900356,106.578870,huyen an duong


In [29]:
street = pd.read_csv("./TripI/street.csv")
street = street.drop(['northeast_lat','northeast_lng','southwest_lat','southwest_lng','distance_from_northeast','distance_from_southwest'], axis = 1)
street.head(3)

,id,district_id,province_id,name,latitude,longitude,name_no_accent,isParsedCorrectly
0,1,667,7,Hưng Bình,18.679468,105.680527,NaN,1
1,2,238,50,Phước Ninh,16.064723,108.220611,NaN,1
2,3,155,20,Phường 1,11.942241,108.433949,NaN,1


# 9. Map hotel_logging with hotel_map

In [30]:
hotel_logging = pd.read_csv("./TripI/Others/hotel_logging.csv")
temp = hotel_map[['domain_hotel_id', 'domain_id','hotel_id_encoded', 'hotel_id' ]]
hotel_logging = hotel_logging.merge(temp, on ='hotel_id')
hotel_logging.head()

C:\Users\ADMIN\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (20) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,id,action_name,user_id,device_id,session_id,ip_address,hotel_id,checkin_date,checkout_date,room_num,...,star_number,reviews_number,rating_level,promo_code,created_at,created_date_id,created_datetime,domain_hotel_id,domain_id,hotel_id_encoded
0,00038961-60dd-4e7b-9f91-62590775ae3b,1_search,10261094556439181748,10046195434681205429,11088181932661319572,58.187.2.44,35590,2020-06-12,2020-06-13,1.0,...,3.0,-1.0,4.0,,1591873502,20200611,2020-06-11 11:05:02,2635002,3.0,9b1f10af-877f-4397-b9a4-d53b3b78cddf
1,08fe3e01-15c2-4f42-8de1-bd702fdd6343,1_search,4570694173250070046,17012117512861306027,15865153089963674129,116.100.204.179,35590,2020-07-12,2020-07-15,1.0,...,3.0,-1.0,4.0,,1594312113,20200709,2020-07-09 16:28:33,2635002,3.0,9b1f10af-877f-4397-b9a4-d53b3b78cddf
2,0aed8407-0892-4833-b136-3c5c1e457e03,1_search,4570694173250070046,17012117512861306027,15865153089963674129,116.100.204.179,35590,2020-07-12,2020-07-15,1.0,...,3.0,-1.0,4.0,,1594312242,20200709,2020-07-09 16:30:42,2635002,3.0,9b1f10af-877f-4397-b9a4-d53b3b78cddf
3,0d1a8d55-cc09-4d71-b14c-da2fe80efd3c,1_search,4570694173250070046,17012117512861306027,15865153089963674129,116.100.204.179,35590,2020-07-12,2020-07-15,1.0,...,3.0,-1.0,4.0,,1594312168,20200709,2020-07-09 16:29:28,2635002,3.0,9b1f10af-877f-4397-b9a4-d53b3b78cddf
4,12f79cbe-ed40-42aa-8f99-02758bac9be5,1_search,4570694173250070046,17012117512861306027,15865153089963674129,116.100.204.179,35590,2020-07-12,2020-07-15,1.0,...,3.0,-1.0,4.0,,1594312262,20200709,2020-07-09 16:31:02,2635002,3.0,9b1f10af-877f-4397-b9a4-d53b3b78cddf


In [31]:
#hotel_logging.to_csv('./hotel_logging.csv', index = False)

# 10. Map hotel_logging with hotel_map

In [32]:
domain_hotel_info = pd.read_csv("./TripI/Others/domain_hotel_info.csv")
temp = hotel_map[['domain_hotel_id','hotel_id_encoded', 'hotel_id' ]]
domain_hotel_info = domain_hotel_info.merge(temp, on ='domain_hotel_id')
domain_hotel_info.head(1)

,id,name,address,url,latitude,longitude,star_number,overall_score,checkin_time,checkout_time,domain_hotel_id,domain_id,domain_name,created_date_id,created_datetime,hotel_id_encoded,hotel_id
0,02786df8-f384-4720-b420-e69c48f9b3ae,YOLO Apartment Homes,"17/5 Tran Bach Dang, Phuoc My, Son Tra, Phước ...",1000001876643,16.0583,108.245,2,-1.0,14:00,12:00,1000001876643,2,Traveloka,20200812,2020-08-12 03:51:30,39f6bd8f-bf85-41a7-8a32-6ab7a17da267,46192.0
